In [169]:
# ruff: noqa: F401, E402, T201, T203, D103

from __future__ import annotations

import math
import sys
import time
from bisect import bisect_left, bisect_right
from collections import Counter, deque
from functools import cmp_to_key, lru_cache, reduce
from heapq import heapify, heappop, heappush, heappushpop, nsmallest
from importlib import reload
from itertools import (
    accumulate,
    chain,
    combinations,
    islice,
    pairwise,
    permutations,
    product,
    repeat,
    starmap,
    tee,
)
from math import comb, exp, factorial, inf, log, prod, sqrt
from operator import add, and_, contains, indexOf, itemgetter, mul, neg, or_, xor
from pprint import pformat, pprint
from typing import Callable, Iterable, Iterator, List, Tuple

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from IPython.display import HTML, clear_output, display

import arrays
import bits
import combinatorics
import graphs
import grid
import lists
import mathematics
import matrix
import parsing
import search
import sequences
import sets
import sorting
import stack
import strings
import trees

for m in (
    arrays,
    bits,
    combinatorics,
    graphs,
    grid,
    lists,
    mathematics,
    matrix,
    parsing,
    search,
    sequences,
    sets,
    sorting,
    stack,
    strings,
    trees,
):
    reload(m)


from combinatorics import fib
from mathematics import prime_numbers as primes
from search import lower_index, upper_index
from sequences import find_if
from strings import splint

np.set_printoptions(edgeitems=5, precision=3, linewidth=300)

In [175]:
from lists import Node


def reverse_groups(n: Node, k: int) -> Node:
    """Reverse the linked list starting at `n` in groups of `k` nodes.

    Parameters
    ----------
    n : Node
        Head node of the linked list.
    k : int
        Number of nodes to be reversed in each group.

    Returns
    -------
    Node
        The head node of the first reversed group.

    """

    def rev(n: Node) -> tuple[Node, Node]:
        """Return: the head of the reversed group, next node after the group."""
        prev = None
        g = k
        while n and g:
            prev, n.next, n = n, prev, n.next
            g -= 1
        return prev, n

    if n and k > 0:
        h, n, prev = *rev(n), n
        while n:
            prev.next, n, prev = *rev(n), n
        return h
    return n

In [176]:
list(reverse_groups(lists.make(range(9)), 0))

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [157]:
list(reverse_groups(lists.make(range(9)), 3))

[2, 1, 0, 5, 4, 3, 8, 7, 6]

In [158]:
list(reverse_groups(lists.make(range(10)), 3))

[2, 1, 0, 5, 4, 3, 8, 7, 6, 9]

In [159]:
list(reverse_groups(lists.make(range(11)), 3))

[2, 1, 0, 5, 4, 3, 8, 7, 6, 10, 9]